In [1]:
import os
#from get_param import params,get_hyperparam,toCuda,toCpu
import torch
from torch.optim import Adam
from torch import nn
import numpy as np

from setups import Dataset


from openfoam.post import *

def save_state(model,optimizer,index="final"):
    """
    saves state of model and optimizer
    :model: model to save (if list: save multiple models)
    :optimizer: optimizer (if list: save multiple optimizers)
    :index: index of state to save (e.g. specific epoch)
    """
    name = "ffffff"
    os.makedirs('Logger/{}/states'.format(name),exist_ok=True)
    path = 'Logger/{}/states/{}.state'.format(name,index)
    state = {}
    
    if type(model)is not list:
        model = [model]
    for i,m in enumerate(model):
        state.update({'model{}'.format(i):m.state_dict()})
    
    if type(optimizer) is not list:
        optimizer = [optimizer]
    for i,o in enumerate(optimizer):
        state.update({'optimizer{}'.format(i):o.state_dict()})
    
    torch.save(state, path)
    

torch.manual_seed(0)
torch.set_num_threads(4)
np.random.seed(0)


width=16
height=16
depth=16
batch_size=10
dataset_size=1

# initialize dataset
dataset = Dataset(width, height, depth, dataset_size)
print("dataset preprocess done")
eps = 0.00000001


loss_multiplier=1


FileNotFoundError: Path './testCase/constant/triSurface/box-in-box.stl' is invalid or does not exist.

In [2]:
from pde_cnn import get_Net

# initialize model
pde_cnn = get_Net().cuda()
pde_cnn.train()

criterion = nn.MSELoss()

# initialize optimizer
optimizer = Adam(pde_cnn.parameters(),lr=0.5)


for epoch in range(191):

	for i in range(1):
		# draw batch from dataset
		mesh_voxel, ref = [xi.cuda() for xi in dataset.fetch(i)]
		
		# apply fluid model on fluid state / boundary conditions for given mu and rho
		#f_velocity, f_pressure, f_kappa = pde_cnn(mesh_voxel)
		ff = pde_cnn(mesh_voxel)

		#a = torch.cat([nn.functional.normalize(f_velocity, p=1, dim=1), nn.functional.normalize(f_pressure, p=1, dim=1), nn.functional.normalize(f_kappa, p=1, dim=1)], dim=1)
		print(ff[:].ravel().shape)
		#loss = torch.sqrt(torch.mean(torch.square(ref - a.ravel())))

		# compute gradients for model parameters
		optimizer.zero_grad()

		#loss = criterion(ff[:].ravel(), ref)

		loss = torch.sum(((ff[:].ravel()- ref) ** 2))
		loss.backward()
		
		# perform optimization step on model
		optimizer.step()
		
		# log losses
		#loss = toCpu(loss).numpy()
		
		if i%1 == 0:
			print(f"{epoch}: i:{i}: loss: {loss.item()/4096.0/5.};")
	
	# save model / optimizer states
	# if True:
	# 	save_state(pde_cnn,optimizer,epoch+1)

		

torch.Size([20480])
0: i:0: loss: 1344.35615234375;
torch.Size([20480])
1: i:0: loss: 1332.0013671875;
torch.Size([20480])
2: i:0: loss: 1297.2677734375;
torch.Size([20480])
3: i:0: loss: 1280.0642578125;
torch.Size([20480])
4: i:0: loss: 1259.63974609375;
torch.Size([20480])
5: i:0: loss: 1237.6390625;
torch.Size([20480])
6: i:0: loss: 1214.9755859375;
torch.Size([20480])
7: i:0: loss: 1192.33017578125;
torch.Size([20480])
8: i:0: loss: 1169.780859375;
torch.Size([20480])
9: i:0: loss: 1146.9189453125;
torch.Size([20480])
10: i:0: loss: 1124.805078125;
torch.Size([20480])
11: i:0: loss: 1103.402734375;
torch.Size([20480])
12: i:0: loss: 1080.75986328125;
torch.Size([20480])
13: i:0: loss: 1057.6259765625;
torch.Size([20480])
14: i:0: loss: 1035.9404296875;
torch.Size([20480])
15: i:0: loss: 1015.23935546875;
torch.Size([20480])
16: i:0: loss: 994.71123046875;
torch.Size([20480])
17: i:0: loss: 974.85087890625;
torch.Size([20480])
18: i:0: loss: 955.3611328125;
torch.Size([20480])
19: 

In [3]:
import pyvista as pv
import cv2

pde_cnn.eval()


mesh_voxel, ref = [xi.cuda() for xi in dataset.fetch(i)]
#f_velocity, f_pressure, f_kappa = pde_cnn(mesh_voxel)
f_velocity = pde_cnn(mesh_voxel)

a = f_velocity[0]


# cv2.namedWindow('v xy',cv2.WINDOW_NORMAL)
# cv2.namedWindow('ref v xy',cv2.WINDOW_NORMAL)
# image = a.cpu().detach().numpy()[3][:, 5, :]/4
# rimage = ref[0].cpu().detach().numpy().reshape(5,16,16,16)[3][:, 5, :]/4

# print(rimage.shape)
# print(rimage)
# #image = cv2.cvtColor(image,cv2.COLOR_HSV2BGR)
# cv2.imshow('ref v xy',rimage)
# cv2.imshow('v xy',image)

# cv2.waitKey(0)

# cv2.destroyAllWindows()
# FILE_PATH = "./testCase/open.foam"
# p = pv.Plotter()
# reader = OpenFOAMReader(FILE_PATH)
# mesh = reader.read()
# internal_mesh = mesh["internalMesh"]  # or internal_mesh = mesh[0]
# p.add_mesh(internal_mesh)
# p.show()


(16, 16)
[[ 3.1593744e+01  3.1243532e+01  2.9605520e+01  2.6569136e+01
   2.2731092e+01  1.7599810e+01  1.3271276e+01  9.3561220e+00
   6.6271019e+00  4.7147427e+00  3.4563160e+00  2.2352030e+00
   1.2435967e+00  7.6357633e-01  4.8454827e-01  1.7929545e-01]
 [ 3.1889076e+01  3.1409868e+01  2.9915598e+01  2.8331667e+01
   2.3708311e+01  1.9666595e+01  1.1780246e+01  8.7489414e+00
   5.4886684e+00  4.2827835e+00  2.5392582e+00  1.8725920e+00
   8.6705202e-01  7.5204766e-01  9.0178847e-01  4.6709916e-01]
 [ 3.3065807e+01  3.2820423e+01  3.1530424e+01  0.0000000e+00
   0.0000000e+00  1.9584593e+01  8.8213282e+00  0.0000000e+00
   0.0000000e+00  2.6469152e+00  1.5681249e+00  0.0000000e+00
   0.0000000e+00  7.4062735e-01  1.3410451e+00  9.9691528e-01]
 [ 3.5290188e+01  3.4320835e+01  0.0000000e+00  0.0000000e+00
   0.0000000e+00  1.2856552e+01 -8.4565562e-01  0.0000000e+00
   0.0000000e+00  5.5143583e-01 -1.2159518e-01  0.0000000e+00
   0.0000000e+00  0.0000000e+00  2.5077496e+00  4.8506851e

In [ ]:


import plotly.graph_objects as go
import numpy as np
X, Y, Z = np.mgrid[-1.5:1.5:16j, 0:3:16j,-1.5:1.5:16j]
image = a[3].cpu().detach().numpy().ravel()


fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=image,
    isomin=0.0,
    isomax=256.18,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=256, # needs to be a large number for good volume rendering
    ))
fig.show()



In [ ]:

fig = go.Figure(data=go.Volume(
    x=X.flatten(),
    y=Y.flatten(),
    z=Z.flatten(),
    value=[max(0.0,-i) for i in mesh_voxel[0][0].cpu().detach().numpy().ravel()],
    isomin=0.0,
    isomax=3.8,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=256, # needs to be a large number for good volume rendering
    ))
fig.show()